## Integration via acceptance-rejection method

A rectangular box is drawn around the integration area, with $a<x<b$ and $0<y<max(f(a),f(b))$, points in the box are randomly sampled, and determined if they are within the desired area.
With enough random points, the numerical value of the integral can be found

In [1]:
import numpy as np
import scipy as sp
from scipy.integrate import quad
from numpy import pi
from scipy.stats.sampling import TransformedDensityRejection

ModuleNotFoundError: No module named 'scipy.stats.sampling'

In [ ]:
# Function to be integrated
def gauss(x):
    y = np.exp(-x**2)
    return y

# Generating points
N = 100000
xmin = 0
xmax = 10
ymin = 0
ymax = max(gauss(xmin),gauss(xmax))

points = np.random.rand(N,2)
points[0:N-1,0] = xmin + (xmax-xmin)*points[0:N-1,0]
points[0:N-1,1] = ymin + (ymax-ymin)*points[0:N-1,1]

# Acceptance-rejection
acc = 0
for i in range(0, N):
    if points[i, 1] < gauss(points[i,0]):
        acc+=1
print("The result from the Monte Carlo algorithm is ", acc/N)

actual = quad(gauss, xmin, xmax)[0]
print("The actual result is ", actual)

However, an acceptance-rejection algorithm with uniform sampling can take very long to converge, dramatically increasing the computation time required to get a good result.

## Integration by the "crude method"

Another method of integration is randomly sampling $N$ points for the interval $a<x<b$ and evaluating the integral as a sum:
$$\int_a^b f(x)\,dx \approx \frac{b-a}{N}\sum_i f(x_i)$$

In [ ]:
N_crude = 100000
xmin = 0
xmax = 10
x_points = xmin + (xmax-xmin)*np.random.rand(N_crude)
y_points = gauss(x_points)

estim = (xmax-xmin)*np.sum(y_points)/N_crude
print(estim)
print("The actual result is ", actual)

This method is also incredibly slow to converge to the actual value. This gives rise to the need for importance sampling.

## Integration by importance sampling

In the case of integrating the Gaussian, points way from $x=0$ rapidly become less significant to the integral. Therefore, sampling more points near $x=0$ will make the sum converge to the actual value more quickly.

For a probability density function $p(x)$ used to generate the samples, the integral is:
$$\int_b^a f(x)\,dx \approx \frac{1}{N}\sum_i\frac{f(x_i)}{p(x_i)}$$

To integrate the Gaussian, the best probability distribution to use is... the Gaussian

In [ ]:
# The Gaussian probability distribution for e^-x^2 from xmin to xmax
urng = np.random.default_rng()
class MyDist:
    def pdf(self, x):
        return np.exp(-x*x)
    def dpdf(self, x):
        return -2*x*np.exp(-x*x)

dist = MyDist()
rng = TransformedDensityRejection(dist, domain=(xmin, xmax),
                                  random_state=urng)


# Sampling
N = 1000
x_points = rng.rvs(N)
p_x = dist(x_points)
y_points = gauss(x_points)

# Evaluating the integral
estim = np.sum(y_points/p_x)/N
print(estim)
print("The actual result is ", actual)